A priori es probable que se sepa qué profesores harán cada clase para cada curso. Esto reduce considerablemente el tamaño del problema, por lo que tiene sentido armar un modelo reducido que considere esto.

In [25]:
import gurobipy as gp
from gurobipy import GRB
from operator import itemgetter
import itertools
import math
import numpy as np 
import json
from prettytable import PrettyTable


class MultiDimensionalArrayEncoder(json.JSONEncoder):
    def encode(self, obj):
        def hint_tuples(item):
            if isinstance(item, tuple):
                return {'__tuple__': True, 'items': item}
            if isinstance(item, list):
                return [hint_tuples(e) for e in item]
            if isinstance(item, dict):
                return {key: hint_tuples(value) for key, value in item.items()}
            else:
                return item

        return super(MultiDimensionalArrayEncoder, self).encode(hint_tuples(obj))
def hinted_tuple_hook(obj):
    if '__tuple__' in obj:
        return tuple(obj['items'])
    else:
        return obj
enc = MultiDimensionalArrayEncoder()

In [18]:
def formar_asignaciones(profesores_por_curso: dict):
    lista_asignaciones = []
    for curso in profesores_por_curso:
        for tupla in profesores_por_curso[curso]:
            lista_asignaciones.append((curso, tupla[0], tupla[1], ))
    return lista_asignaciones 

In [58]:
model = gp.Model("Generación de horarios de colegio")
# model.setParam('OutputFlag', 1)
f = open('parametros2.json',encoding='utf8')

M = 10000
data = json.load(f, object_hook=hinted_tuple_hook)
dias = data['dias']
modulos = data['modulos']
ramos_con_modulos_seguidos = data['ramos_con_modulos_seguidos']
combinaciones = data['combinaciones']
horarios = data['horarios']
profesores_por_curso = data['profesores_por_curso']
ramos_por_curso = data['ramos_por_curso']
# modulos_combinados = data['modulos_combinados']

# ! DATOS HECHIZOS

cantidad_profesores_ramo = {
    "Lenguaje": 10,
    "Matemáticas": 10,
    "Historia": 7,
    "Artes_Visuales": 5,
    "Música": 5,
    "Educación_Física": 8,
    "Orientación": 4,
    "Tecnología": 4,
    "Religión": 5,
    "Ciencias_Naturales": 7,
    "Inglés": 7,
    "Consejo_de_Curso": 1,
    "Filosofía": 1
}

cantidad_cursos_profesor = {
    "Lenguaje": 4,
    "Matemáticas": 4,
    "Historia": 4,
    "Artes_Visuales": 6,
    "Música": 6,
    "Educación_Física": 4,
    "Orientación": 10,
    "Tecnología": 10,
    "Religión": 7,
    "Ciencias_Naturales": 4,
    "Inglés": 4,
    "Consejo_de_Curso": 10,
    "Filosofía": 10
}

import simulador
ramos_por_curso = simulador.generar_horas_libres(ramos_por_curso)
lista_profesores_para_asignaciones, profesores = simulador.generar_profesores(7, 74, cantidad_profesores_ramo, cantidad_cursos_profesor)
profesores_por_curso = simulador.generar_asignaciones(lista_profesores_para_asignaciones, ramos_por_curso)

asignaciones = formar_asignaciones(profesores_por_curso)

x = model.addVars(asignaciones,dias,modulos,vtype=GRB.BINARY, name="x")
s = model.addVars(asignaciones,dias,vtype=GRB.BINARY, name="s")

La clase ocurre en algún momento del horario

In [57]:
model.addConstrs((sum(x[c,p,k,d,j] for p,k in profesores_por_curso[c]) == 1 \
    for c in horarios for d in horarios[c] for j in horarios[c][d]),name="R1");

In [53]:
horarios['1A']['Lunes']

[1, 2, 3, 4, 5, 6, 7]

Cumplir con la cantidad de clases semanales

In [54]:
model.addConstrs((sum(x[c,p,k,d,j] for d in dias for j in modulos) == ramos_por_curso[c][k] \
    for c in horarios for p,k in profesores_por_curso[c]),name="R2");

No pasarse del módulo máximo (se asume que los horarios son continuos)

In [67]:
model.addConstrs((sum(x[curso,p,k,dia,j] for p,k in profesores_por_curso[curso]) == 0 \
    for curso in horarios for dia in horarios[curso] for j in modulos if j > horarios[curso][dia]),name="R3");

Los profesores solo pueden estar en un lugar al mismo tiempo

In [68]:
model.addConstrs((sum(x[c,p,k,dia,j] for c,_,k in asignaciones if _ == p) <= 1 \
    for p in profesores for dia in dias for j in modulos), name="R4");

Módulos simúltaneos (mismo horario) para dos o más cursos

In [81]:
modulos_combinados = [('Educación_Física','7A','7B')]

In [82]:
model.addConstrs(x[c,p,k,dia,j] - x[c2,p,k,dia,j] == 0 for k,c,c2 in modulos_combinados for p in profesores if (p,k) in profesores_por_curso[c] for dia in dias for j in modulos)

KeyError: ('7B', 'Yetgmzbj', 'Educación_Física', 'Lunes', 1)

Módulos de clases seguidos

In [69]:
model.addConstrs(sum(x[curso,p,k,dia,j] for j in modulos) <= 2 for dia in dias for curso in horarios for p,k in profesores_por_curso[curso]);

In [70]:
model.addConstrs(x[c,p,k,dia,j] + x[c,p,k,dia,z] <= 1 for c,p,k in asignaciones for dia in dias for j,z in list(set([(a+1,b+1) for a in range(horarios[c][dia]) for b in range(horarios[c][dia]) if a < b]) - set(combinaciones)));

In [71]:
model.addConstrs((M * s[c,p,k,d] >= sum(x[c,p,k,d,j] for j in modulos) for c,p,k in asignaciones for d in dias),name="RS1");

In [72]:
model.addConstrs((sum(s[curso,p,k,d] for d in dias) <= math.ceil(ramos_por_curso[curso][k] / 2) for curso in horarios for p,k in profesores_por_curso[curso] if k in ramos_con_modulos_seguidos),name="RS2");

In [55]:
model.setObjective(0, GRB.MINIMIZE)

model.write('model.lp')

# model.computeIIS()
# removed =[]
# for c in model.getConstrs():
#     if c.IISConstr:
#         print('%s' % c.constrName)
#         # Remove a single constraint from the model
#         removed.append(str(c.constrName))
#         model.remove(c)

model.update()
model.optimize()
model.write('out.sol')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 190 rows, 1900 columns and 3154 nonzeros
Model fingerprint: 0x48fb0ac5
Variable types: 0 continuous, 1900 integer (1900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 0 rows and 452 columns
Presolve time: 0.01s
Presolved: 190 rows, 1448 columns, 2892 nonzeros
Variable types: 0 continuous, 1448 integer (1444 binary)

Root relaxation: infeasible, 234 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0               - infeasible      -     -    0s

Explored 1 nodes (234 simplex iterations) in 0.02 seconds (0.01 work units)
Thread coun

GurobiError: Unable to retrieve attribute 'X'

In [78]:
def leer_output():
    lista_final = []
    with open('out.sol', 'r') as file:
        lines = file.readlines()
        for line in lines:
            if line[-2] == '1':
                line = line.strip('\n')
                lista_final.append(line)
    # print(lista_final)
    diccionario_dias = {
        'Lunes': 1,
        'Martes': 2,
        'Miércoles': 3,
        'Jueves': 4,
        'Viernes': 5
    }

    vuelta = {
        1: 'Lunes',
        2: 'Martes',
        3: 'Miércoles',
        4: 'Jueves',
        5: 'Viernes'
    }

    profesores_final = []
    for elem in lista_final:
        if elem[0] == "x":
            elem = elem[2:-3]
            elem = elem.split(',')
            profesores_final.append(elem)
    for i in range(len(profesores_final)):
        profesores_final[i][3] = diccionario_dias[profesores_final[i][3]]
    profesores_final = sorted(profesores_final, key= lambda x: x[4])
    profesores_final = sorted(profesores_final, key= lambda x: x[3])
    profesores_final = sorted(profesores_final, key= lambda x: x[0])

    for i in range(len(profesores_final)):
        profesores_final[i][3] = vuelta[profesores_final[i][3]]
    return profesores_final

def generar_horario(curso):

    clases_curso = []
    dias_semana = []
    ultimo_dia = None
    horario = PrettyTable(dias_semana)
    lista_auxiliar = []
    lista_dia = None
    for elem in leer_output():    
        if elem[0] == curso:
            clases_curso.append([elem[2], elem[3], elem[4], elem[1]])
            if elem[3] not in dias_semana:
                dias_semana.append(elem[3])
            if elem[3] != ultimo_dia:
                lista_auxiliar.append(lista_dia)
                lista_dia = []
                ultimo_dia = elem[3]
                lista_dia.append(ultimo_dia)
                elem[2] = elem[2].replace('_', ' ')
                elem[1] = elem[1].replace('_', ' ')
                lista_dia.append(f'{elem[2]} \n {elem[1]}')
            else:
                elem[2] = elem[2].replace('_', ' ')
                elem[1] = elem[1].replace('_', ' ')
                lista_dia.append(f'{elem[2]} \n {elem[1]}')

    lista_auxiliar.append(lista_dia)
    lista_auxiliar = lista_auxiliar[1:]
    len_maxima = 0
    for dia in lista_auxiliar:
        if len(dia) > len_maxima:
            len_maxima = len(dia)

    horario.add_column('', [i+1 for i in range(len_maxima - 1)])

    for i in range(len(lista_auxiliar)):
        for j in range(len_maxima - len(lista_auxiliar[i])):
            lista_auxiliar[i].append('')
        horario.add_column(lista_auxiliar[i][0], lista_auxiliar[i][1:], align='c')
    horario.format = True   
    print(f'Horario del curso {curso}: ')   
    return horario

curso = '8B'
generar_horario(curso)

Horario del curso 8B: 


,Lunes,Martes,Miércoles,Jueves,Viernes
1,Tecnología Ckoktevz,Lenguaje Mgqrxqls,Historia Vcbrisrz,Historia Vcbrisrz,Historia Vcbrisrz
2,Lenguaje Mgqrxqls,Historia Vcbrisrz,Historia Vcbrisrz,Educación Física Glhloygt,Inglés Viguszyr
3,Religión Parbuqwm,Historia Vcbrisrz,Matemáticas Sschojjc,Educación Física Glhloygt,Inglés Viguszyr
4,Historia Vcbrisrz,Matemáticas Sschojjc,Inglés Viguszyr,Lenguaje Mgqrxqls,Matemáticas Sschojjc
5,Inglés Viguszyr,Matemáticas Sschojjc,Tecnología Ckoktevz,Matemáticas Sschojjc,Matemáticas Sschojjc
6,Artes Visuales Cwqcwpow,Ciencias Naturales Hqjqwicc,Ciencias Naturales Hqjqwicc,Orientación Gbqpuktt,Lenguaje Mgqrxqls
7,Artes Visuales Cwqcwpow,Ciencias Naturales Hqjqwicc,Ciencias Naturales Hqjqwicc,Música Nzfcxjga,Lenguaje Mgqrxqls
8,,Religión Parbuqwm,Lenguaje Mgqrxqls,Ciencias Naturales Hqjqwicc,


In [75]:
1/3 + 1/3 + 1/3

1.0